In [1]:
import itertools
from multiprocessing.pool import Pool

import pandas as pd
from sklearn import metrics
from sqlalchemy.orm import Session
from tqdm import tqdm

from april.anomalydetection import BINet
from april.anomalydetection.utils import label_collapse
from april.database import Evaluation
from april.database import Model
from april.database import get_engine
from april.enums import Base
from april.enums import Heuristic
from april.enums import Strategy
from april.evaluator import Evaluator
from april.fs import get_model_files
from april.fs import PLOT_DIR

In [2]:
heuristics = [h for h in Heuristic.keys() if h not in [Heuristic.DEFAULT, Heuristic.MANUAL, Heuristic.RATIO,
                                                       Heuristic.MEDIAN, Heuristic.MEAN]]
params = [(Base.SCORES, Heuristic.DEFAULT, Strategy.SINGLE), *itertools.product([Base.SCORES], heuristics, Strategy.keys())]

In [3]:
def _evaluate(params):
    e, base, heuristic, strategy = params

    session = Session(get_engine())
    model = session.query(Model).filter_by(file_name=e.model_file.name).first()
    session.close()

    if Model is None:
        print(f"Models from session: {model}")
    # Generate evaluation frames
    y_pred = e.binarizer.binarize(base=base, heuristic=heuristic, strategy=strategy, go_backwards=False)
    y_true = e.binarizer.get_targets()

    evaluations = []
    for axis in [0, 1, 2]:
        for i, attribute_name in enumerate(e.dataset.attribute_keys):
            def get_evaluation(label, precision, recall, f1):
                return Evaluation(model_id=model.id, file_name=model.file_name,
                                  label=label, perspective=perspective, attribute_name=attribute_name,
                                  axis=axis, base=base, heuristic=heuristic, strategy=strategy,
                                  precision=precision, recall=recall, f1=f1)

            perspective = 'Control Flow' if i == 0 else 'Data'
            if i > 0 and not e.ad_.supports_attributes:
                evaluations.append(get_evaluation('Normal', 0.0, 0.0, 0.0))
                evaluations.append(get_evaluation('Anomaly', 0.0, 0.0, 0.0))
            else:
                yp = label_collapse(y_pred[:, :, i:i + 1], axis=axis).compressed()
                yt = label_collapse(y_true[:, :, i:i + 1], axis=axis).compressed()
                p, r, f, _ = metrics.precision_recall_fscore_support(yt, yp, labels=[0, 1])
                evaluations.append(get_evaluation('Normal', p[0], r[0], f[0]))
                evaluations.append(get_evaluation('Anomaly', p[1], r[1], f[1]))

    return evaluations

def evaluate(model_name):
    e = Evaluator(model_name)

    _params = []
    for base, heuristic, strategy in params:
        if e.dataset.num_attributes == 1 and strategy in [Strategy.ATTRIBUTE, Strategy.POSITION_ATTRIBUTE]:
            continue
        if isinstance(e.ad_, BINet) and e.ad_.version == 0:
            continue
        if heuristic is not None and heuristic not in e.ad_.supported_heuristics:
            continue
        if strategy is not None and strategy not in e.ad_.supported_strategies:
            continue
        if base is not None and base not in e.ad_.supported_bases:
            continue
        _params.append([e, base, heuristic, strategy])

    return [_e for p in _params for _e in _evaluate(p)]

In [4]:
models = sorted([m.name for m in get_model_files() if m.p == 0.3])# and 'real' in m.name])
print(models)
evaluations = []
with Pool() as p:
    for e in tqdm(p.imap(evaluate, models), total=len(models), desc='Evaluate'):
        evaluations.append(e)

# Write to database
session = Session(get_engine())
for e in evaluations:
    session.bulk_save_objects(e)
    session.commit()
session.close()

['medium-0.3-1_dae_20250319-131452.729609', 'medium-0.3-1_naive_20250319-131450.236340', 'medium-0.3-1_random_20250319-131449.815211', 'medium-0.3-1_sampling_20250319-131450.964413', 'medium-0.3-2_dae_20250319-131521.677698', 'medium-0.3-2_naive_20250319-131450.317506', 'medium-0.3-2_random_20250319-131449.888031', 'medium-0.3-2_sampling_20250319-131451.189182', 'p2p-0.3-1_dae_20250319-131545.154193', 'p2p-0.3-1_naive_20250319-131450.398906', 'p2p-0.3-1_random_20250319-131449.952716', 'p2p-0.3-1_sampling_20250319-131451.416912', 'p2p-0.3-2_dae_20250319-131610.128883', 'p2p-0.3-2_naive_20250319-131450.475005', 'p2p-0.3-2_random_20250319-131450.003953', 'p2p-0.3-2_sampling_20250319-131451.675022', 'paper-0.3-1_dae_20250319-131631.785877', 'paper-0.3-1_naive_20250319-131450.556087', 'paper-0.3-1_random_20250319-131450.063744', 'paper-0.3-1_sampling_20250319-131451.929269', 'small-0.3-1_dae_20250319-131648.279444', 'small-0.3-1_naive_20250319-131450.636780', 'small-0.3-1_random_20250319-13

Evaluate:   0%|          | 0/28 [00:00<?, ?it/s]

Evaluate: 100%|██████████| 28/28 [00:11<00:00,  2.34it/s]


## Pickle the results

In [ ]:
out_dir = PLOT_DIR / 'isj-2019'
eval_file = out_dir / 'eval.pkl'

session = Session(get_engine())
evaluations = session.query(Evaluation).all()
print(vars(evaluations[0]))
rows = []

for ev in tqdm(evaluations):
    print(f"Evaluation: {ev}")
    m = ev.model
    print(f"Model: {m}")
    el = ev.model.training_event_log
    print(f"Event log: {el}")
    rows.append([m.file_name, m.creation_date, m.hyperparameters, m.training_duration, m.training_host, m.algorithm, 
                 el.name, el.base_name, el.percent_anomalies, el.number,
                 ev.axis, ev.base, ev.heuristic, ev.strategy, ev.label, ev.attribute_name, ev.perspective, ev.precision, ev.recall, ev.f1])
session.close()
columns = ['file_name', 'date', 'hyperparameters', 'training_duration', 'training_host', 'ad',
           'dataset_name', 'process_model', 'noise', 'dataset_id',
           'axis', 'base', 'heuristic', 'strategy', 'label', 'attribute_name', 'perspective', 'precision', 'recall', 'f1']
evaluation = pd.DataFrame(rows, columns=columns)

evaluation.to_pickle(eval_file)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7fd016b9bf70>, 'recall': 0.8377298161470823, 'attribute_name': 'name', 'label': 'Normal', 'strategy': 'single', 'base': 'scores', 'file_name': 'medium-0.3-1_dae_20250319-131452.729609.keras', 'id': 1, 'f1': 0.9097222222222222, 'precision': 0.9952516619183286, 'perspective': 'Control Flow', 'heuristic': 'best', 'axis': 0, 'model_id': 1}


100%|██████████| 2754/2754 [00:00<00:00, 25813.12it/s]

Evaluation: <april.database.table.Evaluation object at 0x7fd016b9bac0>
Model: <april.database.table.Model object at 0x7fd03a11b550>
Event log: <april.database.table.EventLog object at 0x7fd018d836a0>
Evaluation: <april.database.table.Evaluation object at 0x7fd016b9b8e0>
Model: <april.database.table.Model object at 0x7fd03a11b550>
Event log: <april.database.table.EventLog object at 0x7fd018d836a0>
Evaluation: <april.database.table.Evaluation object at 0x7fd016b9baf0>
Model: <april.database.table.Model object at 0x7fd03a11b550>
Event log: <april.database.table.EventLog object at 0x7fd018d836a0>
Evaluation: <april.database.table.Evaluation object at 0x7fd016b9b970>
Model: <april.database.table.Model object at 0x7fd03a11b550>
Event log: <april.database.table.EventLog object at 0x7fd018d836a0>
Evaluation: <april.database.table.Evaluation object at 0x7fd016b9b820>
Model: <april.database.table.Model object at 0x7fd03a11b550>
Event log: <april.database.table.EventLog object at 0x7fd018d836a0>


In [6]:
display(evaluation)

,file_name,date,hyperparameters,training_duration,training_host,ad,dataset_name,process_model,noise,dataset_id,axis,base,heuristic,strategy,label,attribute_name,perspective,precision,recall,f1
0,medium-0.3-1_dae_20250319-131452.729609.keras,2025-03-19 13:15:11.812268,None,None,None,dae,medium-0.3-1,medium,0.3,1,0,scores,best,single,Normal,name,Control Flow,0.995252,0.837730,0.909722
1,medium-0.3-1_dae_20250319-131452.729609.keras,2025-03-19 13:15:11.812268,None,None,None,dae,medium-0.3-1,medium,0.3,1,0,scores,best,single,Anomaly,name,Control Flow,0.669202,0.987971,0.797927
2,medium-0.3-1_dae_20250319-131452.729609.keras,2025-03-19 13:15:11.812268,None,None,None,dae,medium-0.3-1,medium,0.3,1,0,scores,best,single,Normal,user,Data,0.905200,1.000000,0.950241
3,medium-0.3-1_dae_20250319-131452.729609.keras,2025-03-19 13:15:11.812268,None,None,None,dae,medium-0.3-1,medium,0.3,1,0,scores,best,single,Anomaly,user,Data,0.000000,0.000000,0.000000
4,medium-0.3-1_dae_20250319-131452.729609.keras,2025-03-19 13:15:11.812268,None,None,None,dae,medium-0.3-1,medium,0.3,1,1,scores,best,single,Normal,name,Control Flow,0.984611,0.925008,0.953879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2749,small-0.3-2_sampling_20250319-131452.466047.keras,2025-03-19 13:14:52.753015,None,None,None,sampling,small-0.3-2,small,0.3,2,2,scores,default,single,Anomaly,name,Control Flow,0.644307,0.704041,0.672851
2750,small-0.3-2_sampling_20250319-131452.466047.keras,2025-03-19 13:14:52.753015,None,None,None,sampling,small-0.3-2,small,0.3,2,2,scores,default,single,Normal,day,Data,0.000000,0.000000,0.000000
2751,small-0.3-2_sampling_20250319-131452.466047.keras,2025-03-19 13:14:52.753015,None,None,None,sampling,small-0.3-2,small,0.3,2,2,scores,default,single,Anomaly,day,Data,0.000000,0.000000,0.000000
2752,small-0.3-2_sampling_20250319-131452.466047.keras,2025-03-19 13:14:52.753015,None,None,None,sampling,small-0.3-2,small,0.3,2,2,scores,default,single,Normal,user,Data,0.000000,0.000000,0.000000
